In [1]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
mnist = input_data.read_data_sets("MNIST_data/" , one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist.train.labels.size

550000

# Variable

In [5]:
X = tf.placeholder(tf.float32 , [None , 28,28,1])
Y = tf.placeholder(tf.float32 , [None , 10])
dropout_prob = tf.placeholder(tf.float32)

# First Layer

In [6]:
W1 = tf.Variable(tf.random_normal([3,3,1,32] , stddev=0.01))
L1 = tf.nn.conv2d(X , W1 , strides=[1,1,1,1] , padding='SAME') # strides = [이미지수,x,y,chaneel]


In [7]:
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1 , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

# Second Layer

In [8]:
W2 = tf.Variable(tf.random_normal([3,3,32,64] , stddev=0.01))
L2 = tf.nn.conv2d(L1, W2 , strides=[1,1,1,1] , padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2 , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

# Third Layer

In [9]:
W3 = tf.Variable(tf.random_normal([7*7*64,256] , stddev=0.01))
L3 = tf.reshape(L2 , [-1 , 7*7*64])
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)

L3 = tf.nn.dropout(L3 , dropout_prob)

# Fourth Layer

In [10]:
W4 = tf.Variable(tf.random_normal([256,10] , stddev=0.01))
model = tf.matmul(L3 , W4)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model , labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [12]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [13]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [14]:
corr = tf.equal(tf.argmax(model , 1) , tf.argmax(Y,1))
acc = tf.reduce_mean(tf.cast(corr, tf.float32))

In [18]:
for epoch in range(2):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs , batch_ys = mnist.train.next_batch(batch_size , shuffle=True)
        batch_xs = batch_xs.reshape(-1,28,28,1)
        
        _,cost_val = sess.run([optimizer , cost] , feed_dict={X:batch_xs , 
                                                              Y : batch_ys,
                                                             dropout_prob:0.5})
        total_cost += cost_val
    print("Epoch : " , "%0.4d" % (epoch+1) , 
         "Avg.Cost = " , "{:,.3f}".format(total_cost/total_batch) , end=' ')
    print('Acc : ' , sess.run(acc , feed_dict={X:mnist.validation.images.reshape(-1,28,28,1) , 
                                          Y: mnist.validation.labels,
                                          dropout_prob:1}))# 이미 inverted drop out 이 되어 있기 때문에 0.5로 하면 안된다.

Epoch :  0001 Avg.Cost =  0.064 Acc :  0.9808
Epoch :  0002 Avg.Cost =  0.053 Acc :  0.9832


In [43]:
batch_xs , batch_ys = mnist.train.next_batch(batch_size)

In [48]:
batch_xs.size

78400

In [52]:
mnist.train.images.size

43120000

In [20]:
corr = tf.equal(tf.argmax(model , 1) , tf.argmax(Y,1))
acc = tf.reduce_mean(tf.cast(corr, tf.float32))
print('Acc : ' , sess.run(acc , feed_dict={X:mnist.test.images.reshape(-1,28,28,1) , 
                                          Y: mnist.test.labels,
                                          dropout_prob:1}))

Acc :  0.9815


In [40]:
corr = tf.equal(tf.argmax(model , 1) , tf.argmax(Y,1))
acc = tf.reduce_mean(tf.cast(corr, tf.float32))
print('Acc : ' , sess.run(acc , feed_dict={X:mnist.test.images.reshape(-1,28,28,1) , 
                                          Y: mnist.test.labels,
                                          dropout_prob:1}))

Acc :  0.9889
